## BERTopic

### Bibliotecas

In [1]:
#Bibliotecas

import pandas as pd
import re
from bertopic import BERTopic
import warnings
import nltk
from nltk.corpus import stopwords
import itertools
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import matplotlib
import matplotlib.pyplot as plt
from umap import UMAP

#Estilizar conteúdo
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def estilo_tabelas(df, max_altura='300px', casas_decimais=2):
    return (
        df.style.set_table_styles(
            [
                {'selector': 'thead th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('border-bottom', '2px solid #007BFF')]},
                {'selector': 'td', 'props': [('font-size', '10px'), ('text-align', 'center'), ('max-height', '40px'), ('white-space', 'nowrap'), ('text-overflow', 'ellipsis'), ('overflow', 'hidden'), ('max-width', '100px')]},
                {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#ffffff')]},
                {'selector': 'table', 'props': [('width', '90%'), ('margin-left', 'auto'), ('margin-right', 'auto'), ('border-collapse', 'collapse')]},
                {'selector': 'td, th', 'props': [('border', '1px solid #666')]},
            ]
        ).set_properties(
            **{'border-color': 'darkgray', 'border-style': 'solid', 'border-width': '1px'}
        ).set_table_attributes(
            f'style="height:auto; overflow:auto; max-height:{max_altura}; display:block;"'  
        ).format(
            precision=casas_decimais  
        )
    )

### Base de dados

In [2]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    decimal=',',
    dtype=dtype)

colunas_descr = ['Status', 'DM_Descricao', 'DA_Descricao', 'PEPL_Descricao', 'PEPR_Descricao', 'DH_Descricao']

df_eventos_descr = df_eventos[colunas_descr].fillna(' ').copy()

### Pré-processamento dos dados

In [3]:
# Baixar stopwords em português
nltk.download('stopwords')
stopwords_pt = stopwords.words('portuguese')

# Função para pré-processamento de texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remover números
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remover caracteres especiais
    text = ' '.join([word for word in text.split() if word not in stopwords_pt])  # Remover stopwords
    return text

# Transformando descrições em listas e removendo strings vazias
lista_dm = [desc for desc in df_eventos_descr['DM_Descricao'].values.tolist() if desc]
lista_da = [desc for desc in df_eventos_descr['DA_Descricao'].values.tolist() if desc]
lista_pepl = [desc for desc in df_eventos_descr['PEPL_Descricao'].values.tolist() if desc]
lista_pepr = [desc for desc in df_eventos_descr['PEPR_Descricao'].values.tolist() if desc]
lista_dh = [desc for desc in df_eventos_descr['DH_Descricao'].values.tolist() if desc]

# Filtrando eventos "Reconhecidos" e "Não reconhecidos"
df_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Reconhecido']
df_nao_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Não reconhecido']

# Criar listas para reconhecidos e não reconhecidos, removendo entradas vazias
lista_dm_r = [desc for desc in df_reconhecido['DM_Descricao'].values.tolist() if desc]
lista_da_r = [desc for desc in df_reconhecido['DA_Descricao'].values.tolist() if desc]
lista_pepl_r = [desc for desc in df_reconhecido['PEPL_Descricao'].values.tolist() if desc]
lista_pepr_r = [desc for desc in df_reconhecido['PEPR_Descricao'].values.tolist() if desc]
lista_dh_r = [desc for desc in df_reconhecido['DH_Descricao'].values.tolist() if desc]

lista_dm_n = [desc for desc in df_nao_reconhecido['DM_Descricao'].values.tolist() if desc]
lista_da_n = [desc for desc in df_nao_reconhecido['DA_Descricao'].values.tolist() if desc]
lista_pepl_n = [desc for desc in df_nao_reconhecido['PEPL_Descricao'].values.tolist() if desc]
lista_pepr_n = [desc for desc in df_nao_reconhecido['PEPR_Descricao'].values.tolist() if desc]
lista_dh_n = [desc for desc in df_nao_reconhecido['DH_Descricao'].values.tolist() if desc]

# Concatenando listas usando itertools.chain
lista_tudo = list(itertools.chain(lista_dm, lista_da, lista_pepl, lista_pepr, lista_dh))
lista_rec_ = list(itertools.chain(lista_dm_r, lista_da_r, lista_pepl_r, lista_pepr_r, lista_dh_r))
lista_nao_ = list(itertools.chain(lista_dm_n, lista_da_n, lista_pepl_n, lista_pepr_n, lista_dh_n))

# Aplicar o pré-processamento
lista_rec = [preprocess_text(text) for text in lista_rec_ if text]
lista_nao = [preprocess_text(text) for text in lista_nao_ if text]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brunamoura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Modelo BERT

#### Tópicos dos eventos reconhecidos

In [4]:
# Eventos reconhecidos
topic_model_rec = BERTopic()
topics_rec, probabilities_rec = topic_model_rec.fit_transform(lista_rec)

# Exibir tópicos identificados para eventos reconhecidos
topic_info_rec = topic_model_rec.get_topic_info()
print("\nTópicos identificados (Reconhecidos):")
print(topic_info_rec)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.



Tópicos identificados (Reconhecidos):
      Topic  Count                                               Name  \
0        -1  48026                        -1_rio_casas_ponte_estradas   
1         0   1071                                0_houvexd_hiuve_xd_   
2         1    955             1_telhas_telhados_telhado_fibrocimento   
3         2    568                   2_materiais_danos_hove_reparties   
4         3    501                    3_banana_macaxeira_bananas_mamo   
5         4    461                          4_linha_km_estrada_metros   
6         5    407         5_motorista_combustvel_motoristas_manuteno   
7         6    386  6_desastres_hidrometeorolgico_fomentando_consu...   
8         7    375                   7_urbana_gargalheiras_zona_tanto   
9         8    374                8_barragem_barragens_vierem_armando   
10        9    352          9_correspondente_ton_referente_quinhentos   
11       10    344               10_danos_mateis_mateirais_ambeintais   
12       11 

In [5]:
# Exibir as palavras-chave de cada tópico reconhecido
for topic_num in topic_info_rec['Topic'].unique():
    if topic_num != -1:  # Ignorar outliers
        print(f"\nTópico {topic_num} (Reconhecidos): {topic_model_rec.get_topic(topic_num)}")


Tópico 0 (Reconhecidos): [('houvexd', np.float64(0.6587827608757515)), ('hiuve', np.float64(0.006466835719750474)), ('xd', np.float64(5.252003746707445e-05)), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]

Tópico 1 (Reconhecidos): [('telhas', np.float64(0.007145414009065248)), ('telhados', np.float64(0.006994406185767663)), ('telhado', np.float64(0.006816469007381564)), ('fibrocimento', np.float64(0.004328845363769761)), ('internet', np.float64(0.0032113466486873923)), ('granizo', np.float64(0.0031218256230036486)), ('telefonia', np.float64(0.00299331409908007)), ('forro', np.float64(0.002829744566939534)), ('cobertura', np.float64(0.0022878652293231803)), ('telecomunicaes', np.float64(0.0022513037521641197))]

Tópico 2 (Reconhecidos): [('materiais', np.float64(0.10526775500116825)), ('danos', np.float64(0.024261035880128144)), ('hove', np.float64(0.0042406740003155955)), ('reparties', np.float64(0.0031951662346183436)), ('aplica', np.float

#### Tópicos dos eventos não reconhecidos

In [6]:
topic_model_nao = BERTopic()
topics_nao, probabilities_nao = topic_model_nao.fit_transform(lista_nao)

# Eventos não reconhecidos
topic_info_nao = topic_model_nao.get_topic_info()
print("\nTópicos identificados (Não reconhecidos):")
print(topic_info_nao)


Tópicos identificados (Não reconhecidos):
     Topic  Count                                               Name  \
0       -1   4706                    -1_municpio_produo_devido_ponte   
1        0    198                    0_estradas_pontes_bueiros_ponte   
2        1    187                        1_rural_zona_populao_potvel   
3        2    171                  2_telhado_telhados_telhas_granizo   
4        3    164                 3_animal_humano_consumo_abasteciam   
5        4    124                      4_escola_escolas_aulas_alunos   
6        5    124                   5_parentes_familias_casa_pessoas   
7        6    121                6_barragens_barragem_poos_cisternas   
8        7    121          7_plantaes_plantio_agricultura_decorrente   
9        8    119             8_casas_casa_habitacionais_desalojadas   
10       9    102                                              9____   
11      10     99             10_ambientais_ambiental_ambiente_danos   
12      11     92    

In [7]:
# Exibir as palavras-chave de cada tópico não reconhecido
for topic_num in topic_info_nao['Topic'].unique():
    if topic_num != -1:  # Ignorar outliers
        print(f"\nTópico {topic_num} (Não reconhecidos): {topic_model_nao.get_topic(topic_num)}")


Tópico 0 (Não reconhecidos): [('estradas', np.float64(0.0262289910866076)), ('pontes', np.float64(0.024648776065250982)), ('bueiros', np.float64(0.019464239533266373)), ('ponte', np.float64(0.015816402930332984)), ('estrada', np.float64(0.013936736552781718)), ('danificadas', np.float64(0.012993200658954745)), ('vicinais', np.float64(0.012310639068294391)), ('pontilhes', np.float64(0.010729304921074148)), ('km', np.float64(0.009867942561109748)), ('linha', np.float64(0.00874908004819946))]

Tópico 1 (Não reconhecidos): [('rural', np.float64(0.025572988478223306)), ('zona', np.float64(0.021739085749213117)), ('populao', np.float64(0.01760345335341059)), ('potvel', np.float64(0.012378127268843525)), ('estiagem', np.float64(0.012168200775391314)), ('abastecimento', np.float64(0.010511897743884998)), ('iomer', np.float64(0.010286580064543461)), ('toda', np.float64(0.010020131072373028)), ('consumo', np.float64(0.009404198794156625)), ('gua', np.float64(0.009333732617863487))]

Tópico 2 (N

### Resultados

#### Eventos reconhecidos

In [8]:
# Visualização dos tópicos
print("\nVisualizando tópicos dos eventos reconhecidos:")
topic_model_rec.visualize_topics()

#### Eventos não reconhecidos

In [9]:
print("\nVisualizando tópicos dos eventos não reconhecidos:")
topic_model_nao.visualize_topics()

Comparar os que erraram e os que acertaram